# The Wonderful World of Data Quality Tools in Python
Sam Bail, Data Umbrella, March 2021

## Imports and data loading
We've got some CSV files with 10,000 row samples of NYC taxi ride data from January and February 2019 which I'm loading here.

In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')

In [4]:
csv1 = 'data/yellow_tripdata_sample_2019-01.csv'
df1 = pd.read_csv(csv1)
df1.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/yellow_tripdata_sample_2019-01.csv'

In [ ]:
csv2 = 'data/yellow_tripdata_sample_2019-02.csv'
df2 = pd.read_csv(csv2)
df2.head()

## Example 1: Pandas "describe" for DataFrames
A simple "profiler" for dataframes. It just gives you some basic statistics on numeric columns.

In [ ]:
df1.describe()

# Example 2: Pandas Profiling
Like a very sophisticated extension of .describe() on Pandas dataframes, 
creates a more detailed profile report of the data.

**Note:** The key difference between the two dataframes is the minimum in the *passenger_count* column:
* In the January data (df1), we have passenger counts from 1 through 6. 
* In the February data (df2), we have counts from 0 through 6, which looks like a bug.

In [ ]:
# Simple way to profile our dataframe and look at the nicely rendered HTML result

from pandas_profiling import ProfileReport
ProfileReport(df1, title="Pandas Profiling Report for df1").to_notebook_iframe()

In [ ]:
ProfileReport(df2, title="Pandas Profiling Report for df2").to_notebook_iframe()

# Example 3: TDDA (Test-Driven Data Analysis)
TDDA allows us to generate "constraints" from a reference data asset and verify whether another data asset matches those constraints.

In [ ]:
# Generate the constraints based on the January dataframe

from tdda.constraints import discover_df, verify_df
constraints = discover_df(df1)
constraints_path = 'tdda_refs/example_constraints.tdda'
with open(constraints_path, 'w') as f:
    f.write(constraints.to_json())
    
# Show the generated constraints
print(str(constraints))

In [ ]:
# Verify that the January data matches the constraints - this should match of course!

v1 = verify_df(df1, constraints_path, type_checking='strict', epsilon=0)
print(str(v1))

In [ ]:
# Verify that the February data matches the constraints - this should fail
# because we have a different min for passenger_count

v2 = verify_df(df2, constraints_path, type_checking='strict', epsilon=0)
print(str(v2))

## Example 4: Bulwark
Data testing framework that lets you add tests on methods that return Pandas dataframes. 
Has some built-in tests and allows custom methods for tests. List of all built-in tests ("checks"): https://bulwark.readthedocs.io/en/stable/bulwark.checks.html

In [ ]:
import bulwark.decorators as dc

# Option 1: Add checks/assertions as decorators on methods that generate dataframes
# This will return the df if all tests pass, and raise errors if any of the tests fail
@dc.HasNoNans()
@dc.IsShape((10000, 5)) # 10000 rows, 5 columns
@dc.HasValsWithinRange(items={"passenger_count": (1,6)}) # min and max are inclusive here
def load_and_test_csv(csv_file_path):
    df = pd.read_csv(csv_file_path)
    return df

# Test this out with the January data
load_and_test_csv(csv1).head()

In [ ]:
# Now let's test the February data - this should fail the "has vals within range" test
load_and_test_csv(csv2).head()

In [ ]:
import bulwark.checks as ck

# Option 2: You can also use the built-in tests ("checks") directly on a dataframe
ck.is_shape(df2, (10000, 5)).head() # 10000 rows, 5 columns
ck.has_vals_within_range(df2, items={"passenger_count": (1,6)}).head()

## Example 5: Voluptous/Opulent Pandas
Voluptous is a data validation library that allows you to specify a "schema" to validate JSON/YAML. 
Opulent Pandas is a df-focused “version” of Voluptuous. The syntax to define and validate a schema
is very similar. 

**Note:** Opulent Pandas doesn't look like it's being actively maintained, so I'm
only showing Voluptuous here.

In [ ]:
from voluptuous import All, Range, ALLOW_EXTRA

# I had to fiddle a little to pass the right dict-type data structure into the schema
# This returns the df if the tests pass, or throws an error if a test fails 
def validate_df(df):
    schema = Schema(
        {
            'vendor_id': All(int)
            'passenger_count': All(int, Range(min=1, max=6))
        }, 
        extra=ALLOW_EXTRA
    )
    for r in df.to_dict('records'):
        schema(r)
    return df

# Test this out with df1, which should pass
validate_df(df1).head()

In [ ]:
validate_df(df2).head()

## Example 6: mobydq

Data validation web app that allows you to check for "indicators" such as completeness, freshness, latency, validity.
Only showing a screenshot here because the setup is pretty heavyweight.

![mobydq](img/mobydq_screenshot.png)

## Example 7: dvc (data version control)
Command-line tool, showing screenshots.


![dvc init](img/dvc_init.png)
![dvc add](img/dvc_add.png)

You can then make modifications to data files and commit new versions and check out previous versions, 
all controlled via the .dvc file. 
dvc also allows you to set remotes to sync data to (instead of managing the actual data via GitHub), e.g. S3.